In [59]:
import os
import sys
sys.path.append("../src")

from Corpus import Corpus

theme = "climatechange"
filename = f"{theme}_corpus_new.csv"

corpus = Corpus(f"Corpus_{theme}")

# --- Si le fichier existe : charger ---
if os.path.exists(filename):
    print(f"\nChargement du corpus depuis {filename}...")
    corpus.load(filename)

# --- Sinon : construire le corpus ---
else:
    print("\nAucun fichier trouvé. Construction du corpus...")

    #  extraction Reddit + Arxiv
    # (les blocs 2 et 3)



Aucun fichier trouvé. Construction du corpus...


In [60]:
import praw
from datetime import datetime

reddit = praw.Reddit(
    client_id='_qZmFflED2oGl3LmqZvqBg',
    client_secret='1wQi5AamyCOBlq5HQqbpu85PDRjySQ',
    user_agent='projet_api'
)

mot_cles = [
    'climate', 'climatechange', 'environment', 'sustainability',
    'carbonfootprint', 'ecology', 'greenenergy', 'climateaction'
]
print("\n--- Extraction Reddit ---")

for mot in mot_cles:
    posts = reddit.subreddit(mot).top(limit=20)
    for post in posts:
        text = post.selftext.replace("\n", " ").strip()
        if text == "":
            continue

        title = post.title
        author_name = str(post.author) if post.author else "Unknown"
        date = datetime.fromtimestamp(post.created_utc).strftime("%Y-%m-%d")
        url = f"https://reddit.com{post.permalink}"
        texte = text
        corpus.add_document(
            post.title,
            str(post.author),
            datetime.fromtimestamp(post.created_utc).strftime("%Y-%m-%d"),
            f"https://reddit.com{post.permalink}",
            texte
        )



--- Extraction Reddit ---


In [ ]:
import urllib.request, xmltodict, ssl, certifi

ssl_context = ssl.create_default_context(cafile=certifi.where())
url = f"https://export.arxiv.org/api/query?search_query=all:{theme}&start=0&max_results=10"

print("\n--- Extraction Arxiv ---")

data = xmltodict.parse(urllib.request.urlopen(url, context=ssl_context).read())
entries = data["feed"]["entry"]
if isinstance(entries, dict):
    entries = [entries]

for entry in entries:
    auteurs = entry["author"]
    if isinstance(auteurs, dict):
        auteurs = [auteurs]
    auteur_str = ", ".join(a["name"] for a in auteurs)

    corpus.add_document(
        entry["title"].strip(),
        auteur_str,
        entry["published"],
        entry["id"],
        entry["summary"].strip()
    )


In [61]:
if not os.path.exists(filename):
    print("\nSauvegarde du corpus...")
    corpus.save(filename)



Sauvegarde du corpus...
Corpus sauvegardé dans climatechange_corpus_new.csv


Tester le corpus avec RedditDocument

In [26]:
import sys
sys.path.append("../src")

from Corpus import Corpus
from RedditDocument import RedditDocument
from ArxivDocument import ArxivDocument

corpus = Corpus("Test")

# Ajout manuel d'un document Reddit
r = RedditDocument(
    titre="Mon post Reddit",
    auteur="Alice",
    date="2024-11-10",
    url="https://reddit.com/r/python",
    texte="Un post intéressant",
    nb_comments=120
)

corpus.id2doc["doc_0"] = r
corpus.ndoc += 1

corpus.afficher_sources()


doc_0 : Reddit - Mon post Reddit


Tester le corpus avec ARXIV

In [27]:
a = ArxivDocument(
    titre="Article d'IA",
    auteur="Yann LeCun",
    date="2023-09-02",
    url="https://arxiv.org/abs/1234.5678",
    texte="Résumé...",
    co_auteurs=["Hinton", "Bengio"]
)

corpus.id2doc["doc_1"] = a
corpus.ndoc += 1
corpus.afficher_sources()


doc_0 : Reddit - Mon post Reddit
doc_1 : Arxiv - Article d'IA


Td6 : test des fonctions 

In [28]:
corpus = Corpus("Test TD6")

corpus.add_document(
    "Climat et pollution",
    "Alice",
    "2024-01-10",
    "http://test1.com",
    "Le changement climatique est rapide. Pollution et climat."
)

corpus.add_document(
    "Energie renouvelable",
    "Bob",
    "2024-01-11",
    "http://test2.com",
    "L'énergie renouvelable lutte contre le changement climatique."
)


In [29]:
corpus.search("climatique")

[('doc_0', <document.Document at 0x1acf48bcd10>),
 ('doc_1', <document.Document at 0x1acf48bc310>)]

In [30]:
for c in corpus.concorde("climatique"):
    print(c)

('Le changement', 'climatique', 'est rapide. Pollution et clim', 'doc_0')
('le lutte contre le changement', 'climatique', '.', 'doc_1')


In [31]:
print("BEFORE:", corpus.id2doc["doc_0"].texte)
corpus.nettoyer_texte()
print("AFTER:", corpus.id2doc["doc_0"].texte)


BEFORE: Le changement climatique est rapide. Pollution et climat.
AFTER: le changement climatique est rapide pollution et climat


In [32]:
corpus.build_vocab()


['changement',
 'climat',
 'climatique',
 'contre',
 'est',
 'et',
 'l',
 'le',
 'lutte',
 'pollution',
 'rapide',
 'renouvelable',
 'énergie']

In [33]:
corpus.term_frequency()


,term_frequency
le,2
changement,2
climatique,2
est,1
rapide,1
pollution,1
et,1
climat,1
l,1
énergie,1


In [34]:
corpus.document_frequency()

,document_frequency
changement,2
climat,1
climatique,2
contre,1
est,1
et,1
l,1
le,2
lutte,1
pollution,1


In [ ]:


print("BEFORE:", corpus.id2doc["doc_0"].texte)
corpus.nettoyer_texte()
print("AFTER:", corpus.id2doc["doc_0"].texte)



[]


,document_frequency
changement,2
climat,1
climatique,2
contre,1
est,1
et,1
l,1
le,2
lutte,1
pollution,1


In [ ]:
import os
import sys
sys.path.append("../src")

from Corpus import Corpus

theme = "climatechange"
filename = f"{theme}_corpus_new.csv"

corpus = Corpus(f"Corpus_{theme}")

# --- Si le fichier existe : charger ---
if os.path.exists(filename):
    print(f"\nChargement du corpus depuis {filename}...")
    corpus.load(filename)

# --- Sinon : construire le corpus ---
else:
    print("\nAucun fichier trouvé. Construction du corpus...")

    #  extraction Reddit + Arxiv
    # (les blocs 2 et 3)



Aucun fichier trouvé. Construction du corpus...


In [ ]:
import praw
from datetime import datetime

reddit = praw.Reddit(
    client_id='_qZmFflED2oGl3LmqZvqBg',
    client_secret='1wQi5AamyCOBlq5HQqbpu85PDRjySQ',
    user_agent='projet_api'
)

mot_cles = [
    'climate', 'climatechange', 'environment', 'sustainability',
    'carbonfootprint', 'ecology', 'greenenergy', 'climateaction'
]
print("\n--- Extraction Reddit ---")

for mot in mot_cles:
    posts = reddit.subreddit(mot).top(limit=20)
    for post in posts:
        text = post.selftext.replace("\n", " ").strip()
        if text == "":
            continue

        title = post.title
        author_name = str(post.author) if post.author else "Unknown"
        date = datetime.fromtimestamp(post.created_utc).strftime("%Y-%m-%d")
        url = f"https://reddit.com{post.permalink}"
        texte = text
        corpus.add_document(
            post.title,
            str(post.author),
            datetime.fromtimestamp(post.created_utc).strftime("%Y-%m-%d"),
            f"https://reddit.com{post.permalink}",
            texte
        )



--- Extraction Reddit ---


In [ ]:
import urllib.request, xmltodict, ssl, certifi

ssl_context = ssl.create_default_context(cafile=certifi.where())
url = f"https://export.arxiv.org/api/query?search_query=all:{theme}&start=0&max_results=10"

print("\n--- Extraction Arxiv ---")

data = xmltodict.parse(urllib.request.urlopen(url, context=ssl_context).read())
entries = data["feed"]["entry"]
if isinstance(entries, dict):
    entries = [entries]

for entry in entries:
    auteurs = entry["author"]
    if isinstance(auteurs, dict):
        auteurs = [auteurs]
    auteur_str = ", ".join(a["name"] for a in auteurs)

    corpus.add_document(
        entry["title"].strip(),
        auteur_str,
        entry["published"],
        entry["id"],
        entry["summary"].strip()
    )


In [ ]:
if not os.path.exists(filename):
    print("\nSauvegarde du corpus...")
    corpus.save(filename)



Sauvegarde du corpus...
Corpus sauvegardé dans climatechange_corpus_new.csv


Tester le corpus avec RedditDocument

In [ ]:
import sys
sys.path.append("../src")

from Corpus import Corpus
from RedditDocument import RedditDocument
from ArxivDocument import ArxivDocument

corpus = Corpus("Test")

# Ajout manuel d'un document Reddit
r = RedditDocument(
    titre="Mon post Reddit",
    auteur="Alice",
    date="2024-11-10",
    url="https://reddit.com/r/python",
    texte="Un post intéressant",
    nb_comments=120
)

corpus.id2doc["doc_0"] = r
corpus.ndoc += 1

corpus.afficher_sources()


doc_0 : Reddit - Mon post Reddit


Tester le corpus avec ARXIV

In [ ]:
a = ArxivDocument(
    titre="Article d'IA",
    auteur="Yann LeCun",
    date="2023-09-02",
    url="https://arxiv.org/abs/1234.5678",
    texte="Résumé...",
    co_auteurs=["Hinton", "Bengio"]
)

corpus.id2doc["doc_1"] = a
corpus.ndoc += 1
corpus.afficher_sources()


doc_0 : Reddit - Mon post Reddit
doc_1 : Arxiv - Article d'IA


Td6 : test des fonctions 

In [ ]:
corpus = Corpus("Test TD6")

corpus.add_document(
    "Climat et pollution",
    "Alice",
    "2024-01-10",
    "http://test1.com",
    "Le changement climatique est rapide. Pollution et climat."
)

corpus.add_document(
    "Energie renouvelable",
    "Bob",
    "2024-01-11",
    "http://test2.com",
    "L'énergie renouvelable lutte contre le changement climatique."
)


In [ ]:
corpus.search("climatique")

[('doc_0', <document.Document at 0x1acf48bcd10>),
 ('doc_1', <document.Document at 0x1acf48bc310>)]

In [ ]:
for c in corpus.concorde("climatique"):
    print(c)

('Le changement', 'climatique', 'est rapide. Pollution et clim', 'doc_0')
('le lutte contre le changement', 'climatique', '.', 'doc_1')


In [ ]:
print("BEFORE:", corpus.id2doc["doc_0"].texte)
corpus.nettoyer_texte()
print("AFTER:", corpus.id2doc["doc_0"].texte)


BEFORE: Le changement climatique est rapide. Pollution et climat.
AFTER: le changement climatique est rapide pollution et climat


In [ ]:
corpus.build_vocab()


['changement',
 'climat',
 'climatique',
 'contre',
 'est',
 'et',
 'l',
 'le',
 'lutte',
 'pollution',
 'rapide',
 'renouvelable',
 'énergie']

In [ ]:
corpus.term_frequency()


,term_frequency
le,2
changement,2
climatique,2
est,1
rapide,1
pollution,1
et,1
climat,1
l,1
énergie,1


In [ ]:
corpus.document_frequency()

,document_frequency
changement,2
climat,1
climatique,2
contre,1
est,1
et,1
l,1
le,2
lutte,1
pollution,1


test sur notre corpus du debut : attention executer 1ere cellule

In [77]:
print("\n===== Pipeline NLP =====")

corpus.nettoyer_texte()

print("\n🔤 Vocabulaire construit...")
corpus.build_vocab()

print("\n📌 Search Results:")
for doc_id, doc in corpus.search("climate")[:5]:
    print(f"{doc_id} → {doc.titre}")

print("\n📌 Concordances:")
for left, mot, right, doc_id in corpus.concorde("climate", window=40)[:10]:
    print(f"...{left} **{mot}** {right}... [{doc_id}]")

print("\n📊 Term Frequency:")
display(corpus.term_frequency().head(20))

print("\n📊 Document Frequency:")
display(corpus.document_frequency().head(20))



===== Pipeline NLP =====

🔤 Vocabulaire construit...

📌 Search Results:
doc_0 → Holy sh*t - for the first time EVER, monsoon winds just crossed the Himalayas into Tibet.
doc_1 → I can absolutely see how we won’t survive increased warming
doc_3 → Joe Rogan Fact-Checks Bernie Sanders on Climate Change Claims - VINnews
doc_18 → Why are people so incapable of understanding the risk of ecological collapse?
doc_27 → Climate Interactive & MIT Launch [Free] Training Program – Exploring Global, Data-driven Climate Solutions and Motivating Effective Climate Action

📌 Concordances:
...called it unprecedented why it matters **climate** change is rewriting global weather sout... [doc_0]
...instream media sleeping on this massive **climate** story anyone tracking it... [doc_0]
...est i always considered my area to be a **climate** haven as it used to be way milder but n... [doc_1]
...rs will produce on critical issues like **climate** change and pandemics rogan completely d... [doc_3]
...ismissed *

,term_frequency
this,36
is,47
absolutely,1
massive,3
monsoon,5
just,9
broke,1
through,5
the,138
himalayas,3



📊 Document Frequency:


,document_frequency
_update,1
a,20
able,1
about,10
absolute,1
absolutely,1
ac,1
accelerating,1
accepted,1
according,1
